# Topic Modeling

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np, pandas as pd
import re, random, os, string

from nltk.tokenize import word_tokenize
from pprint import pprint

### Quick and dirty LSA on small data

In [1]:
TextCorpus = [
"the sky is blue",
"the sun is bright",
"the sun in the sky is bright",
"we can see the shining sun, the bright sun"
]

In [3]:
text_tokens = [word_tokenize(sent) for sent in TextCorpus]
text_tokens

[['the', 'sky', 'is', 'blue'],
 ['the', 'sun', 'is', 'bright'],
 ['the', 'sun', 'in', 'the', 'sky', 'is', 'bright'],
 ['we', 'can', 'see', 'the', 'shining', 'sun', ',', 'the', 'bright', 'sun']]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
transformer = TfidfVectorizer()
tfidf = transformer.fit_transform(TextCorpus) 

In [6]:
tfidf.todense()

matrix([[0.65919112, 0.        , 0.        , 0.        , 0.42075315,
         0.        , 0.        , 0.51971385, 0.        , 0.34399327,
         0.        ],
        [0.        , 0.52210862, 0.        , 0.        , 0.52210862,
         0.        , 0.        , 0.        , 0.52210862, 0.42685801,
         0.        ],
        [0.        , 0.3218464 , 0.        , 0.50423458, 0.3218464 ,
         0.        , 0.        , 0.39754433, 0.3218464 , 0.52626104,
         0.        ],
        [0.        , 0.23910199, 0.37459947, 0.        , 0.        ,
         0.37459947, 0.37459947, 0.        , 0.47820398, 0.39096309,
         0.37459947]])

In [7]:
from sklearn.decomposition import TruncatedSVD

In [8]:
svd = TruncatedSVD(n_components = 3)
lsa = svd.fit_transform(tfidf)

In [9]:
lsa

array([[ 0.62306129,  0.69703778,  0.34250357],
       [ 0.88015639, -0.15398603, -0.30658728],
       [ 0.89278882,  0.12337056, -0.25257634],
       [ 0.67267265, -0.60788755,  0.41913607]])

#### Can measure association using cosine similarity

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
cosine_similarity(lsa[0,:].reshape(1,-1),lsa[1,:].reshape(1,-1))

array([[0.35728045]])

In [13]:
cosine_similarity(lsa)

array([[1.        , 0.35728045, 0.59632771, 0.1397158 ],
       [0.35728045, 1.        , 0.95480174, 0.59048322],
       [0.59632771, 0.95480174, 1.        , 0.44891338],
       [0.1397158 , 0.59048322, 0.44891338, 1.        ]])

## Using LDA package on clean, dummy dataset

In [2]:
import numpy as np
import lda

In [3]:
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()

In [4]:
titles

('0 UK: Prince Charles spearheads British royal revolution. LONDON 1996-08-20',
 '1 GERMANY: Historic Dresden church rising from WW2 ashes. DRESDEN, Germany 1996-08-21',
 "2 INDIA: Mother Teresa's condition said still unstable. CALCUTTA 1996-08-23",
 '3 UK: Palace warns British weekly over Charles pictures. LONDON 1996-08-25',
 '4 INDIA: Mother Teresa, slightly stronger, blesses nuns. CALCUTTA 1996-08-25',
 "5 INDIA: Mother Teresa's condition unchanged, thousands pray. CALCUTTA 1996-08-25",
 '6 INDIA: Mother Teresa shows signs of strength, blesses nuns. CALCUTTA 1996-08-26',
 "7 INDIA: Mother Teresa's condition improves, many pray. CALCUTTA, India 1996-08-25",
 '8 INDIA: Mother Teresa improves, nuns pray for "miracle". CALCUTTA 1996-08-26',
 '9 UK: Charles under fire over prospect of Queen Camilla. LONDON 1996-08-26',
 '10 UK: Britain tells Charles to forget Camilla. LONDON 1996-08-27',
 "11 COTE D'IVOIRE: FEATURE - Quiet homecoming for reprieved Ivory Coast maid. ABIDJAN 1996-08-28",


In [5]:
X.shape

(395, 4258)

In [6]:
?lda.LDA

In [7]:
%%time
model = lda.LDA(n_topics=20, n_iter=500, random_state=1)
model.fit(X)

INFO:lda:n_documents: 395
INFO:lda:vocab_size: 4258
INFO:lda:n_words: 84010
INFO:lda:n_topics: 20
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -1051748
INFO:lda:<10> log likelihood: -719800
INFO:lda:<20> log likelihood: -699115
INFO:lda:<30> log likelihood: -689370
INFO:lda:<40> log likelihood: -684918
INFO:lda:<50> log likelihood: -681322
INFO:lda:<60> log likelihood: -678979
INFO:lda:<70> log likelihood: -676598
INFO:lda:<80> log likelihood: -675383
INFO:lda:<90> log likelihood: -673316
INFO:lda:<100> log likelihood: -672761
INFO:lda:<110> log likelihood: -671320
INFO:lda:<120> log likelihood: -669744
INFO:lda:<130> log likelihood: -669292
INFO:lda:<140> log likelihood: -667940
INFO:lda:<150> log likelihood: -668038
INFO:lda:<160> log likelihood: -667429
INFO:lda:<170> log likelihood: -666475
INFO:lda:<180> log likelihood: -665562
INFO:lda:<190> log likelihood: -664920
INFO:lda:<200> log likelihood: -664979
INFO:lda:<210> log likelihood: -664722
INFO:lda:<220> log likelihood: -6

Wall time: 6.51 s


In [8]:
topic_word = model.topic_word_

In [10]:
len(topic_word[0])

4258

In [11]:
n_top_words = 10

In [12]:
[np.argsort(topic_word[0])][:-n_top_words:-1]

[array([   0, 2773, 2774, ...,   32,   24,   36], dtype=int64)]

In [38]:
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
     print(" ".join(topic_words))

Topic 0: government british minister west group letters party former million
Topic 1: church first during people political country ceremony visit government
Topic 2: elvis king wright fans presley concert life death first
Topic 3: yeltsin russian russia president kremlin michael romania orthodox operation
Topic 4: pope vatican paul surgery pontiff john hospital trip rome
Topic 5: family police miami versace cunanan funeral home church kennedy
Topic 6: south simpson born york white north african black wife
Topic 7: order church mother successor since election religious head nuns
Topic 8: charles prince diana royal queen king parker bowles camilla
Topic 9: film france french against actor paris bardot magazine poster
Topic 10: germany german war nazi christian letter book scientology jews
Topic 11: east prize peace timor quebec belo indonesia nobel award
Topic 12: n't told life people church show very public come
Topic 13: years world time year last say three later news
Topic 14: mother 

### Amazon Tap reviews data

In [11]:
df = pd.read_csv('Tap Reviews.csv')

In [12]:
df.head()

,asins,name,reviews.numHelpful,reviews.rating,reviews.text,reviews.title
0,B01BH83OOM,Amazon Tap - Alexa-Enabled Portable Bluetooth ...,806,5,IMPORTANT UPDATE (3/8/17): As you read my orig...,"The Echo's Little Sister Is AMAZING!! 3,"
1,B01BH83OOM,Amazon Tap - Alexa-Enabled Portable Bluetooth ...,426,5,It was just a few weeks ago that I was bemoani...,Tap Alexa on the go!
2,B01BH83OOM,Amazon Tap - Alexa-Enabled Portable Bluetooth ...,295,5,The Amazon Tap is a portable Bluetooth speaker...,Amazon Tap Review - Best Bluetooth Speaker in ...
3,B01BH83OOM,Amazon Tap - Alexa-Enabled Portable Bluetooth ...,0,5,Look at this product as a portable speaker fir...,Great for what it does
4,B01BH83OOM,Amazon Tap - Alexa-Enabled Portable Bluetooth ...,3,5,This Amazon tap is not only a great Bluetooth ...,"Awesome, smart little portable speaker"


In [23]:
df = df.rename(columns={'reviews.text':'review'})

In [25]:
df.columns

Index(['asins', 'name', 'reviews.numHelpful', 'reviews.rating', 'review',
       'reviews.title'],
      dtype='object')

In [24]:
df.shape

(542, 6)

#### Text pre-processing

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') + list(string.punctuation)

In [17]:
from gensim.utils import simple_preprocess

In [18]:
simple_preprocess("I don't like this product!")

['don', 'like', 'this', 'product']

In [19]:
?simple_preprocess

In [20]:
word_tokenize("I don't like this product!")

['I', 'do', "n't", 'like', 'this', 'product', '!']

**Important Note:** Models are not sutomatically downloaded with Spacy, so you may need to do a ```python -m spacy download en``` to use its preprocessing methods.

In [ ]:
# gensim for LDA 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# create dictionary and corpus
# Create Dictionary
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[2])

### Building the Topic Model

In [14]:
from gensim.models import ldamodel

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [15]:
?ldamodel.LdaModel

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

In [244]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.100*"use" + 0.055*"love" + 0.045*"easy" + 0.044*"great" + 0.042*"buy" + '
  '0.029*"excellent" + 0.028*"gift" + 0.028*"enjoy" + 0.019*"work" + '
  '0.016*"product"'),
 (1,
  '0.052*"echo" + 0.047*"alexa" + 0.031*"button" + 0.030*"tap" + 0.026*"great" '
  '+ 0.022*"hand" + 0.022*"talk" + 0.019*"portable" + 0.018*"sound" + '
  '0.018*"set"'),
 (2,
  '0.055*"phone" + 0.027*"love" + 0.021*"light" + 0.021*"information" + '
  '0.020*"move" + 0.018*"old" + 0.018*"month" + 0.017*"apple" + 0.016*"app" + '
  '0.015*"system"'),
 (3,
  '0.036*"button" + 0.028*"size" + 0.024*"alexa" + 0.022*"speaker" + '
  '0.021*"loud" + 0.020*"dot" + 0.020*"press" + 0.019*"put" + 0.019*"get" + '
  '0.017*"echo"'),
 (4,
  '0.044*"question" + 0.035*"need" + 0.030*"get" + 0.029*"time" + 0.028*"ask" '
  '+ 0.027*"lot" + 0.023*"love" + 0.022*"pretty" + 0.020*"anywhere" + '
  '0.020*"answer"'),
 (5,
  '0.036*"fun" + 0.032*"carry" + 0.021*"super" + 0.019*"high" + 0.017*"news" + '
  '0.017*"case" + 0.016*"clock

In [ ]:
# Compute Perplexity
print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [246]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      42.207699        1       1 -0.264039  0.041807
0      10.291842        1       2 -0.022344  0.253717
1       9.701098        1       3 -0.191751 -0.059601
3       8.575901        1       4 -0.107875 -0.169075
4       8.266050        1       5  0.028820  0.015968
6       5.955149        1       6  0.024296 -0.110216
2       4.601311        1       7  0.080099  0.017682
7       3.869910        1       8  0.072324  0.123882
5       3.482035        1       9  0.206849 -0.039021
8       3.049007        1      10  0.173622 -0.075143, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
99    Default  175.000000          use  175.000000  30.0000  30.0000
89    Default  276.000000          tap  276.000000  29.0000  29.0000
29    Default  224.000000         echo  224.000000  28.0000  28.0000
177   Default  221.000000        great  221.000000  27.0000  27.0000
55    Default  146.000000         love  146.000000  26.0000  26.0000
1     Default  168.000000        alexa  168.000000  25.0000  25.0000
42    Default  162.000000         good  162.000000  24.0000  24.0000
417   Default   58.000000         easy   58.000000  23.0000  23.0000
39    Default  113.000000          get  113.000000  22.0000  22.0000
163   Default   66.000000       button   66.000000  21.0000  21.0000
86    Default  209.000000        sound  209.000000  20.0000  20.0000
71    Default   35.000000     question   35.000000  19.0000  19.0000
270   Default   33.000000        phone   33.000000  18.0000  18.0000
13    Default   92.000000          buy   92.000000  17.0000  17.0000
304   Default   47.000000  portability   47.000000  16.0000  16.0000
136   Default   26.000000        price   26.000000  15.0000  15.0000
94    Default   57.000000         time   57.000000  14.0000  14.0000
378   Default   36.000000         need   36.000000  13.0000  13.0000
530   Default   26.000000          fun   26.000000  12.0000  12.0000
447   Default   18.000000         wish   18.000000  11.0000  11.0000
286   Default   38.000000          ask   38.000000  10.0000  10.0000
205   Default  195.000000      speaker  195.000000   9.0000   9.0000
339   Default   28.000000    excellent   28.000000   8.0000   8.0000
195   Default   96.000000     portable   96.000000   7.0000   7.0000
25    Default   50.000000          dot   50.000000   6.0000   6.0000
3     Default  116.000000       amazon  116.000000   5.0000   5.0000
57    Default   36.000000         make   36.000000   4.0000   4.0000
366   Default   30.000000         gift   30.000000   3.0000   3.0000
313   Default   39.000000     purchase   39.000000   2.0000   2.0000
503   Default   23.000000         size   23.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
738   Topic10    3.513580      charger    4.251817   3.2996  -4.4006
445   Topic10    3.387434          son    4.125440   3.2933  -4.4372
829   Topic10    2.999254       simple    3.737487   3.2703  -4.5589
1427  Topic10    2.599464           yr    3.337501   3.2404  -4.7020
1426  Topic10    2.599464           us    3.337501   3.2404  -4.7020
1425  Topic10    2.599464       moment    3.337501   3.2404  -4.7020
1424  Topic10    2.599464         dual    3.337501   3.2404  -4.7020
826   Topic10    2.410651        cover    3.148663   3.2233  -4.7774
830   Topic10    2.410564        usage    3.148700   3.2232  -4.7774
1013  Topic10    2.085135        drive    2.823146   3.1873  -4.9224
814   Topic10    2.056512     enjoying    2.794600   3.1837  -4.9362
768   Topic10    1.972889         step    2.710883   3.1726  -4.9778
940   Topic10    1.882040        score    2.620050   3.1595  -5.0249
941   Topic10    1.882040        sport    2.620050   3.1595  -5.0249
1028  Topic10    1.881977     basement    2.620064   3.1595  -5.02

In [243]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=42,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### Get topics for any new document

In [ ]:
new_doc = "enter text here"

In [ ]:
new_doc_bow = diction.doc2bow(new_doc)
new_doc_bow

In [ ]:
lda_mod.get_document_topics(new_doc_bow)

### Choosing the number of topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []

    for num_topics in range(start, limit, step):
        model_list.append(model)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, 
                                           random_state=42, passes=10, per_word_topics=True)
        coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherence_model_lda.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=10, step=2)

In [ ]:
limit=40; start=2; step=6;
x = range(start, limit, step)

for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))